## Task 1 – Data Gathering and Pre-processing


### API Connection

#### Importing libraries and setting keys

In [1]:
import tweepy
API_KEY = 'yNrkYGayTA4tqIRWfNmwd350n'
API_SECRET_KEY = 'MA8EOw0hJp55gjWrwupYjOdmuTRxBvgpaal7lycwobNAqnEckx'
ACCESS_TOKEN = '1342791046317670400-AeXR52Ohn483w4wzQQ6Zgwzef8Cx3T'
ACCESS_TOKEN_SECRET = 'g1W4dKjZrFlHlvkMOWLlBQ7PtcIAFpRAgm4gaoYMoAfDg'
BEARER_TOKEN = 'AAAAAAAAAAAAAAAAAAAAAExyUgEAAAAAdgqhXDb9kFhloRV35jEE50HTeUY%3DXRvwQyJDaYvYHM5NHHWPpCDxUt7CsmJfdoAQHPA9m0SKQL3dY6'

#### Establishing connection with Twitter

In [2]:
auth = tweepy.OAuthHandler(API_KEY, API_SECRET_KEY)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)


#### Getting the users data

In [32]:
# Because the API can only give us the first 200 and we need 300 we need to be a bit clever about it a do 2 requests one that
# will return 200 and one that will return 100 bellow the last one
# We use 101 as the count because the max_id we be included which afterwards we are going to drop
# Lastly we combine the 200 and the 100 timelines to get 300 tweets
billGatesUser = api.get_user(screen_name='BillGates')
billGatesTimeline200 = api.user_timeline(screen_name='BillGates', count = 200, tweet_mode='extended')### Needs to be added in everything
billGatesTimeline100 = api.user_timeline(screen_name='BillGates', count = 101, max_id = billGatesTimeline200[199]._json['id'])[1:101]
billGatesTimeline = billGatesTimeline200 + billGatesTimeline100

elonMuskUser = api.get_user(screen_name='elonmusk')
elonMuskTimeline200 = api.user_timeline(screen_name='elonmusk', count = 200)
elonMuskTimeline100 = api.user_timeline(screen_name='elonmusk', count = 101, max_id = elonMuskTimeline200[199]._json['id'])[1:101]
elonMuskTimeline = elonMuskTimeline200 + elonMuskTimeline100


ellenDeGeneresUser = api.get_user(screen_name='TheEllenShow')
ellenDeGeneresTimeline200 = api.user_timeline(screen_name='TheEllenShow', count = 200)
ellenDeGeneresTimeline100 = api.user_timeline(screen_name='TheEllenShow', count = 101, max_id = ellenDeGeneresTimeline200[199]._json['id'])[1:101]
ellenDeGeneresTimeline = ellenDeGeneresTimeline200 + ellenDeGeneresTimeline100

### Data Pre-processing

#### Necessary imports

In [33]:
import pandas as pd
import json

#### Definition of a function for the timeline processing 

In [34]:
def setUpTimelineList(timeline):
    listOfTweets = []
    for each_json_tweet in timeline:
        listOfTweets.append(each_json_tweet._json)
    return listOfTweets

#### Bill Gates pre-processing

In [35]:
billGatesUserFrame = pd.json_normalize(billGatesUser._json)

billGatesTimelineFrame = pd.DataFrame(setUpTimelineList(billGatesTimeline))

#### Elon Musk pre-processing

In [36]:
elonMuskUserFrame = pd.json_normalize(elonMuskUser._json)

elonMuskTimelineFrame = pd.DataFrame(setUpTimelineList(elonMuskTimeline))

#### Ellen DeGenerous pre-processing

In [37]:
ellenDeGeneresUserFrame = pd.json_normalize(ellenDeGeneresUser._json)

ellenDeGeneresTimelineFrame = pd.DataFrame(setUpTimelineList(ellenDeGeneresTimeline))

#### Adding all the users together into one dataframe

In [38]:
frames = [billGatesUserFrame, elonMuskUserFrame, ellenDeGeneresUserFrame]
usersFrame = pd.concat(frames)

### Data Cleansing 

#### Definition a function for field selection

In [39]:
def cleanUpColumns(columnsToKeep, dataframe):
    for column in dataframe:
        if column not in columnsToKeep:
            dataframe = dataframe.drop([column], axis = 1)
    return dataframe

#### Removing all the unnecessary data from the usersFrame

In [40]:
#Just to get a bit clever we will just write the ones we want 
importantColumns = ['id', 'name', 'screen_name', 'description', 'followers_count', 'friends_count', 'listed_count', 'created_at', 
                    'favourites_count', 'statuses_count', 'status.id', 'status.text']
usersFrame = cleanUpColumns(importantColumns, usersFrame)

#### Removing all the unnecessary data from the users timelines frames

In [41]:
#Following the same process as above we write only the onces we need
importantColumns = ['created_at', 'id', 'text','truncated', 'entities', 'in_reply_to_status_id','in_reply_to_user_id',
                    'in_reply_to_screen_name', 'is_quote_status','retweet_count', 'favorite_count']
billGatesTimelineFrame = cleanUpColumns(importantColumns, billGatesTimelineFrame)
elonMuskTimelineFrame = cleanUpColumns(importantColumns, elonMuskTimelineFrame)
ellenDeGeneresTimelineFrame = cleanUpColumns(importantColumns, ellenDeGeneresTimelineFrame)

#### Creating columns for hashtags and users mentioned from entities

In [42]:
# The bellow functions are used to process the hashtags and the users mentioned from the entities columns  
def processingHashtags(entity):
    if entity['hashtags']:
        hashtags = []
        for hashtag in entity['hashtags']:
            hashtags.append(hashtag['text'])
        return hashtags
    return 'N/A'

def processingUserMentioned(entity):
    if entity['user_mentions']:
        users = []
        for user in entity['user_mentions']:
            users.append(user['screen_name'])
        return users
    return 'N/A'

In [43]:
#The bellow function extracts the important data from the entities column and then it drops the column
def processEntities(dataframe):
    dataframe['hashtags'] = dataframe['entities'].apply(processingHashtags)
    dataframe['user_mentions'] = dataframe['entities'].apply(processingUserMentioned)
    dataframe = dataframe.drop(['entities'], axis=1)
    return dataframe

In [44]:
billGatesTimelineFrame = processEntities(billGatesTimelineFrame)
elonMuskTimelineFrame = processEntities(elonMuskTimelineFrame)
ellenDeGeneresTimelineFrame = processEntities(ellenDeGeneresTimelineFrame)

#### Converting all the NaN's to N/A

In [45]:
billGatesTimelineFrame = billGatesTimelineFrame.fillna('N/A')
elonMuskTimelineFrame = elonMuskTimelineFrame.fillna('N/A')
ellenDeGeneresTimelineFrame = ellenDeGeneresTimelineFrame.fillna('N/A')
usersFrame = usersFrame.fillna('N/A')


#### Fixing the dates

In [46]:
billGatesTimelineFrame['created_at'] = pd.to_datetime(billGatesTimelineFrame['created_at'])
elonMuskTimelineFrame['created_at'] = pd.to_datetime(elonMuskTimelineFrame['created_at'])
ellenDeGeneresTimelineFrame['created_at'] = pd.to_datetime(ellenDeGeneresTimelineFrame['created_at'])
usersFrame['created_at'] = pd.to_datetime(usersFrame['created_at'])

In [49]:
billGatesTimeline[0]._json['full_text']

'Thank you, @SecGranholm for your leadership. The commitment of @ENERGY and the innovate spirit of our National Labs will drive the breakthroughs we need to help the whole world get to #NetZero. https://t.co/RutwW0EFoH'

#### Thigs to note

Even though the data cleansing will be easier and faster with the tweeter API V2 we decided to use the V1 because we fill that it has more appropriate data for our research.

#### Analysis on users mentioned

In [18]:
def getUsersMentioned(timelineFrame):
    users = {}
    for listOfUsers in timelineFrame['user_mentions']:
        if listOfUsers != 'N/A':
            for user in listOfUsers:
                if user in users:
                    users[user] +=1
                else:
                    users[user] = 1
    return pd.Series(users)

In [19]:
bill = getUsersMentioned(billGatesTimelineFrame)
elon = getUsersMentioned(elonMuskTimelineFrame)
ellen = getUsersMentioned(ellenDeGeneresTimelineFrame)

In [20]:
bill.sort_values(ascending=False).head(10)

Breakthrough       17
gatesfoundation     9
GlobalFund          5
melindagates        5
narendramodi        4
Rotary              3
andersoncooper      3
POTUS               3
rajpanjabi          3
MSuzman             3
dtype: int64

In [21]:
billGatesTimelineFrame['user_mentions'].value_counts().head(1) / 300 * 100

N/A    55.666667
Name: user_mentions, dtype: float64

In [22]:
ellen.sort_values(ascending=False).head(10)

OhBubble           12
KristenBell        11
KimKardashian       9
nbc                 7
official_tWitch     5
daxshepard          5
TheEllenShow        5
TiffanyHaddish      4
mariolopezviva      4
ArsenioHall         3
dtype: int64

In [23]:
ellenDeGeneresTimelineFrame['user_mentions'].value_counts().head(1) / 300 * 100

N/A    52.666667
Name: user_mentions, dtype: float64

In [24]:
elon.sort_values(ascending=False).head(10) 

Tesla             22
SpaceX            16
BillyM2k          15
teslaownersSV     15
WholeMarsBlog     13
Teslarati         12
PPathole          11
SawyerMerritt     10
Erdayastronaut     7
WR4NYGov           6
dtype: int64

In [25]:
elonMuskTimelineFrame['user_mentions'].value_counts().head(1) / 300 * 100

N/A    12.333333
Name: user_mentions, dtype: float64

We can see that both Elon and Bill mainly mention their companies/foundations while Ellen mostly mentions the people that appear in her show. Bill has the lowest users mentioned in his tweets with 42% tweets with users mentioned and Elon the most with only 13% of his tweets not having a user mentioned.  

In [27]:
type(billGatesTimelineFrame['created_at'][0])

str